In [148]:
# imports
import json
import urllib.request


In [149]:
# open the training file
url = "https://raw.githubusercontent.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection/master/Sarcasm_Headlines_Dataset.json"
# url = "https://raw.githubusercontent.com/lin-justin/sarcasm-detection/master/Sarcasm_Headlines_Dataset_v2.json"

# need to massage the text to put it into proper json format
with urllib.request.urlopen(url) as url_json:
    json_text = "["

    for line in url_json:
        line_string = line.decode()
#        print("got line with type: {}".format(type(line_string))
        json_text = json_text + line_string + ","

    json_text = json_text[:-1] + "]"
    file_data = json.loads(json_text)
    print("got data of type: {}".format(type(url_json)))
    print("got json data of type: {}".format(type(json_text)))


got data of type: <class 'http.client.HTTPResponse'>
got json data of type: <class 'str'>


In [150]:
# parse the json
sentence_list = []
label_list = []
url_list = []

for item in file_data:
    sentence_list.append(item['headline'])
    label_list.append(item['is_sarcastic'])
    url_list.append(item['article_link'])

print("got {} elements in my training data".format(len(sentence_list)))


got 28619 elements in my training data


In [151]:
# tokenize the text data for the NN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# tokenize
tokenizer = Tokenizer(oov_token = '<OOV>')
tokenizer.fit_on_texts(sentence_list)

# get the word index
word_index = tokenizer.word_index
print("got word index of size: {}".format(len(word_index)))



got word index of size: 30885


In [152]:
# get the sentence word sequences
sequences = tokenizer.texts_to_sequences(sentence_list)

# pad tomake sure all row sequnces have the same length and print
padded_sequences = pad_sequences(sequences, padding = 'post')
print("the first sequence is: {}".format(padded_sequences[0]))
print("the shape of the padded seuqnces is: {} with type: {}".format(padded_sequences.shape, type(padded_sequences)))

the first sequence is: [16004   355  3167  7474  2644     3   661  1119     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
the shape of the padded seuqnces is: (28619, 152) w

In [153]:
# split into train and test sets
# specify training split percent
train_split = 0.85

# get the index for the split
train_index = int(train_split * padded_sequences.shape[0])
print("the traim row index is: {}".format(train_index))



the traim row index is: 24326


In [154]:
# split into train and test
feature_train = padded_sequences[0: train_index]
label_train = label_list[0: train_index]
feature_test = padded_sequences[train_index:]
label_test = label_list[train_index:]

print("split data into train of size {} and test of size {}".format(len(label_train), len(label_test)))

split data into train of size 24326 and test of size 4293


In [32]:
# now need to create a new training tokenizer fit only on the traim data
tokenizer_train = Tokenizer(oov_token='<OOV\>')

# split the train and test senetences
sentence_train = sentence_list[:train_index]
sentence_test = sentence_list[train_index:]

# train this tokenizer on the training features
tokenizer_train.fit_on_texts(sentence_train)

train_word_index = tokenizer_train.word_index
print("the train sentences word in is of size {}".format(len(train_word_index)))


the train sentences word in is of size 28488


In [33]:
# now get the sequences for the train and test sentence lists
train_sequence = tokenizer_train.texts_to_sequences(sentence_train)
test_sequence = tokenizer_train.texts_to_sequences(sentence_test)

# pad (TODO: parameterize the constants)
train_sequence_padded = pad_sequences(train_sequence, padding='post', truncating='post', maxlen=120)
test_sequence_padded = pad_sequences(test_sequence, padding='post', truncating='post', maxlen=120)

# make sure the shapes match on the column dimension (needed for the NN)
print("the train padded sequence is of shape {} and the test padded sequence is of shape {}".format(train_sequence_padded.shape, test_sequence_padded.shape))

the train padded sequence is of shape (24326, 120) and the test padded sequence is of shape (4293, 120)


In [147]:
train_word_index

NameError: name 'train_word_index' is not defined

In [43]:
import tensorflow as tf 

# build the NN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(train_word_index), 16, input_length = 120),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print("the tf model summary is {}".format(model.summary()))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 16)           455808    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 25        
Total params: 456,241
Trainable params: 456,241
Non-trainable params: 0
_________________________________________________________________
the tf model summary is None


In [44]:
# train the tf model
num_epochs = 50

model.fit(train_sequence_padded, label_train, epochs= num_epochs, validation_data= (test_sequence_padded, label_test), verbose= 2)



ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})